In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score, precision_recall_curve, auc
from scipy.stats import entropy
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from scipy.stats import entropy



In [2]:
import sys
sys.path.append('/home/cmottez/CS231N_Lightweight_Bias_Mitigation_Chest_Xray')

from metrics import compute_metrics, compute_kl_divergence_sex, compute_kl_divergence_age, compute_kl_divergence_race, compute_metrics_subcath, bias_table, bias_table_auprc


### Read the CSV with your extracted embeddings and the demographics info

In [ ]:
train = pd.read_pickle('../CNN_from_scratch/final_embeddings_train.pkl')
valid = pd.read_pickle('../CNN_from_scratch/final_embeddings_val.pkl')
test = pd.read_pickle('../CNN_from_scratch/final_embeddings_test.pkl')

In [4]:
train.head()

,sex,race,age,Cardiomegaly,Edema,Lung Opacity,Pleural Effusion,path_to_image,embedding_0,embedding_1,...,embedding_1014,embedding_1015,embedding_1016,embedding_1017,embedding_1018,embedding_1019,embedding_1020,embedding_1021,embedding_1022,embedding_1023
0,0,0,0,0,0,0,1,train/patient02424/study8/view1_frontal.jpg,0.577020,0.225139,...,0.026545,0.039251,0.103660,0.115058,0.078481,0.210349,0.213531,0.023333,0.020806,0.067287
1,0,1,0,1,1,1,0,train/patient08517/study10/view1_frontal.jpg,0.131081,0.408154,...,0.058802,0.018378,0.000000,1.004865,0.690274,0.037327,0.000000,0.140191,0.386501,0.005471
2,0,0,1,0,0,1,1,train/patient03164/study14/view1_frontal.jpg,0.036016,0.260678,...,0.000021,0.000000,0.000000,0.007108,0.123461,0.089736,0.258684,0.001910,0.161937,0.000000
3,1,0,0,0,0,0,0,train/patient28576/study3/view1_frontal.jpg,0.498498,0.238422,...,0.344112,0.363110,0.349447,0.067082,0.008733,0.033510,0.091146,0.307771,0.309890,0.467222
4,1,0,0,0,0,0,1,train/patient35066/study5/view1_frontal.jpg,0.203171,0.254078,...,0.048831,0.106363,0.082369,0.000000,0.153965,0.548223,0.739487,0.015420,0.073128,0.070378


### Train test

In [5]:
train_embeddings = train[[f"embedding_{i}" for i in range(1024)]]
valid_embeddings = valid[[f"embedding_{i}" for i in range(1024)]]
test_embeddings = test[[f"embedding_{i}" for i in range(1024)]]

# Diseases to predict
diseases = ['Cardiomegaly', 'Lung Opacity', 'Edema', 'Pleural Effusion']


# Labels for train and test
y_train = train[diseases]
y_test = test[diseases]

X_train = train_embeddings
X_test = test_embeddings


In [6]:
y_sex = test['sex']
y_race = test['race']
y_age = test['age']


y_train_sex = train['sex']
y_train_race = train['race']
y_train_age = train['age']
group_info = train[['sex','race','age']]

In [7]:
y_train.replace(-1, 0, inplace=True)
y_test.replace(-1, 0, inplace=True)

/tmp/ipykernel_751471/3958011541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.replace(-1, 0, inplace=True)
/tmp/ipykernel_751471/3958011541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test.replace(-1, 0, inplace=True)


### PCA to reduce embeddings

In [ ]:
# Step 1: Standardize the embeddings_list to have mean 0 and variance 1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(np.stack(train_embeddings.values))
X_test_scaled = scaler.transform(np.stack(test_embeddings.values))

# Step 2: Set target variance threshold (e.g., 95%)
variance_threshold = 0.95

# Step 3: Fit PCA to determine the optimal number of components based on variance threshold
pca_full = PCA()
pca_full.fit(X_train_scaled)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Step 4: Find the number of components that meets the variance threshold
optimal_components = np.argmax(cumulative_variance >= variance_threshold) + 1
print(f"Optimal number of components to retain {variance_threshold*100}% variance: {optimal_components}")

#95% variance means that the selected principal components (reduced dimensions) retain 95% of the total variability present in the original high-dimensional data.

Optimal number of components to retain 95.0% variance: 30


In [ ]:
# Apply PCA if wanted
pca = PCA(n_components=optimal_components)
x_train_subset = pca.fit_transform(X_train_scaled)
x_test_subset = pca.transform(X_test_scaled)

In [10]:
y_train.shape

(67263, 4)

In [11]:
x_pool = x_train_subset
y_pool = y_train

In [12]:
def train_model(x_train, y_train, x_test, y_test, model):
    
    if y_train.shape[1] > 1:
        multi_output_model = MultiOutputClassifier(model)
        multi_output_model.fit(x_train, y_train)

        if hasattr(model, "predict_proba"):
            y_test_preds_proba = pd.DataFrame({disease: probs[:, 1] for disease, probs in zip(diseases, multi_output_model.predict_proba(x_test))}) # Dataframe with probabilites 
        else:
            y_test_preds_proba = None

    else:
        print("Single disease")
        model.fit(x_train, y_train)
        if hasattr(model, "predict_proba"):
            # Dataframe with probabilities for the positive class
            y_test_preds_proba = pd.DataFrame(model.predict_proba(x_test)[:, 1], columns=['Probability'])
        else:
            y_test_preds_proba = None


    return y_test_preds_proba 


In [ ]:
def compute_uncertainty(probas):
    # example: margin sampling for binary: |p-0.5|
    # for multi-class use: smallest difference between top two probs
    if probas.shape[1] == 2:
        return 1.0 - np.abs(probas[:,1] - 0.5) * 2
    else:
        top2 = np.partition(probas, -2, axis=1)[:, -2:]
        return 1.0 - (top2[:,-1] - top2[:,-2])

def active_learn(
    x_pool,                # unlabeled features pool (DataFrame or array)
    y_pool,                # true labels for simulation (array or DataFrame)
    group_pool,            # DataFrame with group info (sex/race/age), used for underrepresentation
    x_seed, y_seed,        # initial small labeled set
    model, 
    batch_size=50, 
    n_rounds=10,
    diversity_clusters=10
):
    x_labeled, y_labeled, grp_labeled = x_seed.copy(), y_seed.copy(), group_pool.loc[y_seed.index]
    x_unlabeled, y_unlabeled, grp_unlabeled = x_pool.copy(), y_pool.copy(), group_pool.copy()
    
    for round in range(n_rounds):
        # 1) Fit
        if isinstance(y_labeled, pd.DataFrame) and y_labeled.shape[1] > 1:
            clf = MultiOutputClassifier(model)
            clf.fit(x_labeled, y_labeled)
        else:
            clf = model
            clf.fit(x_labeled, y_labeled.values.ravel())
        
        # 2) Uncertainty on pool
        if hasattr(clf, "predict_proba"):

            if isinstance(clf, MultiOutputClassifier):
                probs = np.stack([est.predict_proba(x_unlabeled) 
                                  for est in clf.estimators_], axis=2)
                
                uncs = np.mean([compute_uncertainty(probs[:,:,t]) 
                                for t in range(probs.shape[2])], axis=0)
            else:
                probs = clf.predict_proba(x_unlabeled)
                uncs = compute_uncertainty(probs)
        else:
            raise ValueError("Model has no predict_proba – implement a margin or entropy on decision_function")
        
            # 3) Under‐representation weighting
        # Build a single “group key” (sex_race_age) for labeled & unlabeled
        def make_key(df):
            return df.astype(str).agg('_'.join, axis=1)

        labeled_keys   = make_key(grp_labeled)
        unlabeled_keys = make_key(grp_unlabeled)

        # Compute frequency of each group in the labeled set
        freq = labeled_keys.value_counts(normalize=True)

        # Map each unlabeled sample to weight = 1/freq(group)
        # If a new combination somehow appears, fall back to the smallest freq
        min_freq = freq.min()
        weights = unlabeled_keys.map(lambda k: 1.0 / freq.get(k, min_freq))

        # Now your sample_scores line works as expected:
        sample_scores = uncs * weights.values

        # 4) Diversity clustering: pick top B*cluster_factor and cluster into B clusters
        topk = np.argsort(sample_scores)[-batch_size * diversity_clusters:]
        kmeans = KMeans(n_clusters=batch_size, random_state=round).fit(x_unlabeled.iloc[topk])
        # pick one from each cluster: the most uncertain in that cluster
        new_idx = []
        for c in range(batch_size):
            members = np.where(kmeans.labels_ == c)[0]
            cluster_idxs = topk[members]
            # pick highest score
            pick = cluster_idxs[np.argmax(sample_scores[cluster_idxs])]
            new_idx.append(pick)
        
        # 5) Add to labeled, remove from pool
        x_new = x_unlabeled.iloc[new_idx]
        y_new = y_unlabeled.iloc[new_idx]
        grp_new = grp_unlabeled.iloc[new_idx]
        
        x_labeled = pd.concat([x_labeled, x_new], axis=0)
        y_labeled = pd.concat([y_labeled, y_new], axis=0)
        grp_labeled = pd.concat([grp_labeled, grp_new], axis=0)
        
        x_unlabeled = x_unlabeled.drop(x_new.index)
        y_unlabeled = y_unlabeled.drop(y_new.index)
        grp_unlabeled = grp_unlabeled.drop(grp_new.index)
        
        print(f"Round {round+1}/{n_rounds}: added {len(new_idx)} samples, labeled set size = {len(x_labeled)}")
    
    return clf, x_labeled, y_labeled


In [14]:
x_pool = pd.DataFrame(x_pool)
y_pool = pd.DataFrame(y_pool)

In [15]:
to_keep = 15000
x_seed, y_seed = x_pool[:to_keep], y_pool[:to_keep]

# unlabeled pool is the rest
x_pool2 = x_pool[to_keep:]
y_pool2 = y_pool[to_keep:]
group_pool2 = group_info.iloc[to_keep:]

In [16]:
x_pool2.reset_index(drop=True, inplace=True)
y_pool2.reset_index(drop=True, inplace=True)
group_pool2.reset_index(drop=True, inplace=True)

In [ ]:
def train_model(x_train, y_train, x_test, model):
    """
    Fits (or refits) your model on (x_train, y_train) and returns a DataFrame of 
    per-disease probability-for-class-1 on x_test.
    """

    is_multi = (hasattr(y_train, "shape") and y_train.shape[-1] > 1)

    if is_multi:
        if isinstance(model, MultiOutputClassifier):
            m = model
        else:
            m = MultiOutputClassifier(model)
        m.fit(x_train, y_train)
        
        proba_lists = m.predict_proba(x_test)
        y_test_preds_proba = pd.DataFrame({
            disease: probs[:, 1]
            for disease, probs in zip(diseases, proba_lists)
        })
    else:
        if not hasattr(model, "predict_proba"):
            raise ValueError("Model must support predict_proba for single‐output mode")
        model.fit(x_train, y_train.ravel())
        y_test_preds_proba = pd.DataFrame(
            model.predict_proba(x_test)[:, 1],
            columns=["Probability"]
        )

    return m, y_test_preds_proba


In [ ]:
dataframes = []

for i in range(5):

    xgb = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss',
    learning_rate=0.05,
    n_estimators=150,
    max_depth=10,
    subsample=1.0,
    colsample_bytree=0.6,
    reg_lambda=5,
    reg_alpha=5,
    scale_pos_weight=5,
    random_state=i
    )

    learner, X_lab, Y_lab = active_learn(
        x_pool2, y_pool2, group_pool2,
        x_seed, y_seed,
        model=xgb,
        batch_size=2000,
        n_rounds=10,
        diversity_clusters=10
    )

    X_lab.reset_index(drop=True, inplace=True)
    Y_lab.reset_index(drop=True, inplace=True)


    # final evaluation on held-out x_test_subset, y_test as before
    learner, y_pred = train_model(
        x_train = X_lab,
        y_train = Y_lab,
        x_test  = x_test_subset,
        model   = learner 
    )

    predictions = y_pred.values
    targets = y_test.values

    y_pred_df = pd.DataFrame(y_pred)
    y_pred_df.columns = diseases
    y_test_df = pd.DataFrame(y_test)
    y_test_df.columns = diseases

    metrics = compute_metrics(y_pred_df, y_test_df, diseases)
    metrics_female, metrics_male, metrics_white, metrics_black, metrics_asian, metrics_young, metrics_old = compute_metrics_subcath(predictions, targets, diseases, y_sex, y_race, y_age)
    styled_df, df = bias_table_auprc(metrics, metrics_female, metrics_male, metrics_white, metrics_black, metrics_asian, metrics_young, metrics_old)

    dataframes.append(df)

In [ ]:
combined_df = pd.concat(dataframes, keys=range(len(dataframes)))

combined_df = combined_df[['AUPRC', 'AUC', 'Delta AUPRC sex', 'Delta AUPRC race', 'Delta AUPRC age']]

df_mean = combined_df.groupby(level=1).mean()
df_std = combined_df.groupby(level=1).std() 

df_mean.reset_index(drop=True, inplace=True)
df_std.reset_index(drop=True, inplace=True)

df_mean.insert(0, 'disease', diseases)
df_std.insert(0, 'disease', diseases)
